## Import libraries

In [1]:
from pathlib import Path
import pandas as pd
import openpyxl
import math

## Access input and output directories 

In [2]:
#Access input folder
input_dir1 = Path ("Pressure_curves_excel_with_MinMaxTime")
print ("1",input_dir1)

#Access folders inside input folder
input_dir2 =  [folder for folder in input_dir1.iterdir() if folder.is_dir()]
print ("2",input_dir2)

1 Pressure_curves_excel_with_MinMaxTime
2 [WindowsPath('Pressure_curves_excel_with_MinMaxTime/L13_1'), WindowsPath('Pressure_curves_excel_with_MinMaxTime/L13_11'), WindowsPath('Pressure_curves_excel_with_MinMaxTime/L13_13'), WindowsPath('Pressure_curves_excel_with_MinMaxTime/L13_15'), WindowsPath('Pressure_curves_excel_with_MinMaxTime/L13_17'), WindowsPath('Pressure_curves_excel_with_MinMaxTime/L13_19'), WindowsPath('Pressure_curves_excel_with_MinMaxTime/L13_21'), WindowsPath('Pressure_curves_excel_with_MinMaxTime/L13_23'), WindowsPath('Pressure_curves_excel_with_MinMaxTime/L13_25'), WindowsPath('Pressure_curves_excel_with_MinMaxTime/L13_27'), WindowsPath('Pressure_curves_excel_with_MinMaxTime/L13_29'), WindowsPath('Pressure_curves_excel_with_MinMaxTime/L13_3'), WindowsPath('Pressure_curves_excel_with_MinMaxTime/L13_31'), WindowsPath('Pressure_curves_excel_with_MinMaxTime/L13_33'), WindowsPath('Pressure_curves_excel_with_MinMaxTime/L13_35'), WindowsPath('Pressure_curves_excel_with_MinM

## Define find_change_time function

In [3]:
def find_change_time (Max_index, sh):
                 
    for j in range(Max_index, sh.max_row+1):
        cell_pressure = sh.cell (row = j, column = 2)
        if float(cell_pressure.value) < 101200:
            print ("Change_pressure", cell_pressure.value)
            change_time = sh.cell (row = j, column = 1)
            print ("Change_time_other", change_time.value)
            change_index = j
            break
    
    return change_time, change_index

## Filtering change time

In [6]:
# Create empty dataframe
df = pd.DataFrame(columns = ['Charge_mass', 'Standoff_distance', 'Change_time', 'Change_index'])

# Access folders
for folder in input_dir2:
    print ("folder", folder)
    
    # Set charge mass
    charge_mass = int(folder.stem[4:])/2
    
    # Make a list of data file names
    files = list(folder.rglob("*.xlsx"))
    
    # Access files inside folder
    for file in files:
        print (file.stem)
    
        wrkbk = openpyxl.load_workbook(file)
        sh = wrkbk.active
        
        # Set standoff distance
        stand_dis = math.ceil(int(file.stem[1:])/2)
        
        Max_time = sh.cell(row = 2, column = 4).value
        print ("Max_time_file_defined", Max_time, type(Max_time))
        
        Max_index = 2
        print ("Max_index = 2", Max_index)
            
        for i in range(2, sh.max_row+1):
            Time = sh.cell (row = i, column = 1)
            if Time.value >= Max_time:
                print ("Time > Max_time", Time.value)
                Max_index = i
                print ("Max_time_row after loop", i)
                break
            
        change_time, change_index = find_change_time (Max_index, sh)
        
        # append rows to an empty DataFrame
        new_row = {'Charge_mass' : [charge_mass],
                   'Standoff_distance' : [stand_dis],
                   'Change_time' : [change_time.value],
                   'Change_index' : [change_index]}
        new_row_data = pd.DataFrame(new_row)
        df = pd.concat([df, new_row_data], ignore_index = True)

folder Pressure_curves_excel_with_MinMaxTime\L13_1
P1
Max_time_file_defined 0.0006695743 <class 'float'>
Max_index = 2 2
Time > Max_time 0.00066957430681
Max_time_row after loop 69
Change_pressure 101051.85156
Change_time_other 0.0012998265447
P13
Max_time_file_defined 0.0158798229 <class 'float'>
Max_index = 2 2
Time > Max_time 0.015889702365
Max_time_row after loop 1591
Change_pressure 101188.8125
Change_time_other 0.018559759483
P17
Max_time_file_defined 0.0216297656 <class 'float'>
Max_index = 2 2
Time > Max_time 0.02162976563
Max_time_row after loop 2165
Change_pressure 101183.05469
Change_time_other 0.024529954419
P21
Max_time_file_defined 0.0274298247 <class 'float'>
Max_index = 2 2
Time > Max_time 0.027439706028
Max_time_row after loop 2746
Change_pressure 101193.25781
Change_time_other 0.03049980849
P25
Max_time_file_defined 0.0332799591 <class 'float'>
Max_index = 2 2
Time > Max_time 0.033279959112
Max_time_row after loop 3330
Change_pressure 101192.46094
Change_time_other 0.

In [7]:
print (df.head(20))

    Charge_mass Standoff_distance  Change_time Change_index
0           0.5                 1      0.00130          132
1           0.5                 7      0.01856         1858
2           0.5                 9      0.02453         2455
3           0.5                11      0.03050         3052
4           0.5                13      0.03648         3650
5           0.5                15      0.04246         4248
6           0.5                17      0.04844         4846
7           0.5                19      0.05443         5445
8           0.5                21      0.06041         6043
9           0.5                23      0.06639         6641
10          0.5                25      0.07238         7240
11          0.5                 3      0.00672          674
12          0.5                27      0.07837         7839
13          0.5                29      0.08435         8437
14          0.5                30      0.08735         8737
15          0.5                 5      0

## Create End_time excel file

In [9]:
with pd.ExcelWriter('Change_time.xlsx', engine = "openpyxl", mode='a', if_sheet_exists='replace') as writer:  
    df.to_excel(writer, sheet_name = "Sheet1", index = False, header = True)